## Implementation of Skip Connections (ResNet Architecture)

In [0]:
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras import datasets
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, AveragePooling2D, MaxPooling2D, add, concatenate
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import to_categorical

### Load CIFAR10 dataset

In [0]:
data = datasets.cifar10.load_data()
((x_train, y_train), (x_test, y_test)) = data
print(x_train.shape, x_test.shape)

(50000, 32, 32, 3) (10000, 32, 32, 3)


In [0]:
x_train = x_train.astype('float') / 255
x_test = x_test.astype('float') / 255
num_classes = len(np.unique(y_train))
print("Number of classes = ", num_classes)

Number of classes =  10


In [0]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

### Model Construction

In [0]:
input_img = Input(shape=x_train.shape[1:])
print(input_img.shape)

# layer 1
layer = Conv2D(16, (3, 3), padding='same', name = "from_input_conv")(input_img)
x = BatchNormalization(name = "from_input_batch")(layer)
# x = Activation('relu')(layer)
numfilters = 16

for stack in range(3):
  print("Stack", stack)
  for block in range(3):
    print("Block", block)
    strides = 1
    if stack>0 and block==0:
      strides = 2
  
    name = "resnet"+str(stack)+"_"+str(block)+"_"
    # layer 2
    layer = Conv2D(numfilters, (3,3), strides = strides, padding='same', name = name+"1conv")(x)
    layer = BatchNormalization(name = name+"1_batch")(layer)
    y = Activation('relu', name =name + "1_acti")(layer)

    # layer 3
    layer = Conv2D(numfilters, (3,3), padding='same', name =name+"2_conv")(y)
    y = BatchNormalization(name = name+"2_batch")(layer)
    # y = Activation('relu')(layer)
    if stack > 0 and block == 0:
      x = Conv2D(numfilters, (1,1), padding='same', strides = strides, name =name+"skip")(x)

    x = add([y, x])
    x = Activation('relu')(x)
  numfilters *= 2


(?, 32, 32, 3)
Stack 0
Block 0
Block 1
Block 2
Stack 1
Block 0
Block 1
Block 2
Stack 2
Block 0
Block 1
Block 2


In [0]:
x = AveragePooling2D(pool_size=8,padding="same")(x)

y = Flatten()(x)
# x = Dense(2018)(x)
# x = Activation('relu')(x)
# x = Dense(512)(x)
# x = Activation('relu')(x)
y = Dense(10)(y)
op = Activation('softmax')(y)

In [0]:
model = Model(inputs=input_img, outputs=op)

In [0]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
from_input_conv (Conv2D)        (None, 32, 32, 16)   448         input_9[0][0]                    
__________________________________________________________________________________________________
from_input_batch (BatchNormaliz (None, 32, 32, 16)   64          from_input_conv[0][0]            
__________________________________________________________________________________________________
resnet0_0_1conv (Conv2D)        (None, 32, 32, 16)   2320        from_input_batch[0][0]           
____________________________________________________________________________________________

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
esCallback = EarlyStopping(monitor='val_loss',
                             min_delta=0,
                             patience=0,
                             verbose=0,
                             mode='min')
history = model.fit(x_train,y_train,
              epochs=20,
              validation_split=0.1,
              shuffle=True,verbose=1)

Train on 45000 samples, validate on 5000 samples
Epoch 1/20
45000/45000 [==============================] - 21s 462us/sample - loss: 1.5883 - acc: 0.4141 - val_loss: 1.4921 - val_acc: 0.4802
Epoch 2/20
45000/45000 [==============================] - 19s 418us/sample - loss: 1.2283 - acc: 0.5572 - val_loss: 1.4210 - val_acc: 0.5080
Epoch 3/20
45000/45000 [==============================] - 19s 429us/sample - loss: 1.0469 - acc: 0.6230 - val_loss: 1.4287 - val_acc: 0.5202
Epoch 4/20
45000/45000 [==============================] - 19s 418us/sample - loss: 0.9281 - acc: 0.6681 - val_loss: 1.0445 - val_acc: 0.6280
Epoch 5/20
45000/45000 [==============================] - 19s 420us/sample - loss: 0.8400 - acc: 0.7025 - val_loss: 1.0400 - val_acc: 0.6334
Epoch 6/20
45000/45000 [==============================] - 19s 417us/sample - loss: 0.7705 - acc: 0.7279 - val_loss: 0.9026 - val_acc: 0.6810
Epoch 7/20
45000/45000 [==============================] - 19s 418us/sample - loss: 0.7050 - acc: 0.7530 -

In [0]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 159us/sample - loss: 1.5643 - acc: 0.6457
Test loss: 1.5642582077026368
Test accuracy: 0.6457
